# Исследование надежности заемщиков.

# Описание проекта
В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
# открываем файл для просмотра
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('D:\IT\courses Python\yandex_practicum\\2_project_preprocessing\data.csv')

data.head()
data.info() # есть пропуски в столбцах total_income, days_employed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Применим цикл для каждого столбца таблицы. Найдем количество всех пропусков. Выведем на экран проценты, поделив на общее количество

In [3]:
for column in data: 
    row = data[column].isna().sum()
    print('{} : {:.0%}'.format(column, row / len(data))) # 10% составляют пропуски в столбцах days_employed, total_income

children : 0%
days_employed : 10%
dob_years : 0%
education : 0%
education_id : 0%
family_status : 0%
family_status_id : 0%
gender : 0%
income_type : 0%
debt : 0%
total_income : 10%
purpose : 0%


Доход, который без справки 2ндфл- это тоже самое что пропуск в значениях. Меня больше смущает точный % пропусков в 2х столбцах. Как будто это системная ошибка, хорошо бы уточнить у программистов.

Стаж и зарплату правильнее считать по середине данных- медианой. 

In [4]:
data['days_employed'] = data['days_employed'].fillna(value=(data['days_employed'].median())) # Заменила все пропуски медианным значением
display(data['days_employed'].head())

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
Name: days_employed, dtype: float64

In [5]:
data['total_income'] = data['total_income'].fillna(value=(data['total_income'].median())) # Здесь тоже самое
display(data['total_income'].head())

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
Name: total_income, dtype: float64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Начну проверку по порядку с первого столбца

In [6]:
data['children'] = data['children'].abs() # Убираю отрицательное значение в количестве детей

In [7]:
data['days_employed'] = data['days_employed'].abs() # Преобразую данные стажа в столбце days_employed в положительное значение.
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [8]:
data['education'].value_counts() # Много одинаковых слов по разному написаны.

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [9]:
data['education'] = data['education'].str.lower() # Все привела в нижний регистр и присвоила новые значения

In [10]:
data['education'].value_counts() # Все ок

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [11]:
data['gender'].value_counts() # Здесь есть одно значение непонятного пола.

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'  # Заменим значение XNA на F

In [13]:
data['gender'].value_counts() # Проверяю чистоту. Все ок

F    14237
M     7288
Name: gender, dtype: int64

In [14]:
data['purpose'].value_counts() # Здесь много идентичных целей, но написаных поразному. 

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Шаг 2.3. Изменение типов данных.

In [15]:
data['total_income'] = data['total_income'].astype('int') # Для получения целых чисел, применила astype()

In [16]:
data.describe() # Составила статистическую таблицу. Пропусков нет, все значения определены. 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.543275,60277.957929,43.293380,0.817236,0.972544,0.080883,1.651589e+05
std,1.379876,133301.583103,12.574584,0.548138,1.420324,0.272661,9.786608e+04
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1025.608174,33.000000,1.000000,0.000000,0.000000,1.077980e+05
50%,0.000000,1808.053434,42.000000,1.000000,0.000000,0.000000,1.450170e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955430e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Шаг 2.4. Удаление дубликатов.

In [17]:
data.duplicated().sum() # Сколько всего дубликатов? 71

71

Пропусков нет. 
Аномалии обработаны. 
К единому реестру приведены. 
Итого 71 дубль. 

In [18]:
data = data.drop_duplicates().reset_index(drop=True)   # Удаляю дубликаты. И заменила старый датафрейм на новый без дубликатов.
data.duplicated().sum()  

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [19]:
data_education = data[['education_id', 'education']]  # Создадим новый датафрейм из двух столбцов
data_education.tail()

,education_id,education
21449,1,среднее
21450,1,среднее
21451,1,среднее
21452,1,среднее
21453,1,среднее


In [20]:
data_family_status = data[['family_status_id', 'family_status']]
data_family_status.tail()

,family_status_id,family_status
21449,1,гражданский брак
21450,0,женат / замужем
21451,1,гражданский брак
21452,0,женат / замужем
21453,0,женат / замужем


In [21]:
data.drop(['education', 'family_status'], axis=1).head()  # Удаляем столбцы у существующей таблицы и выводим первые 5 строк

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [22]:
def total_income_category(row): # Создали функцию для разбивки доходов по категориям
    if row <= 30000:
        return 'E'
    if row <= 50000:
        return 'D'
    if row <= 200000:
        return 'C'
    if row <= 1000000:
        return 'B'
    return 'A'

In [23]:
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head(6)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B


### Шаг 2.7. Категоризация целей кредита.

In [24]:
def data_purpose(row):  # Создали функцию для уменьшения дубликатов
    if 'недвижимост' in row:
        return 'операции с недвижимостью'
    if 'образован' in row:
        return 'получение образования'
    if 'автомобил' in row:
        return 'операции с автомобилем'
    if 'свадьб' in row:
        return 'проведение свадьбы'
    return 'операции с недвижимостью'

In [25]:
data['purpose_category'] = data['purpose'].apply(data_purpose) # В новом столбце применили функцию
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [26]:
data['purpose_category'].value_counts() # Все отсортировано

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

### Ответы на вопросы.

1) Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создам сводную таблицу. Видно, что при увеличении количества детей, растет доля должников. 

In [27]:
data_children_debt = data.pivot_table(index='children', values='debt', aggfunc=('count', 'sum', 'mean')) 
# С круглыми скобками в aggfunc названия столбцов получается одноуровневыми, без названия значений values.
display(data_children_debt)

,count,mean,sum
children,,,
0,14091,0.075438,1063
1,4855,0.091658,445
2,2052,0.094542,194
3,330,0.081818,27
4,41,0.097561,4
5,9,0.000000,0
20,76,0.105263,8


In [28]:
data_children_debt['share_deptors'] = data_children_debt['mean']*100
display(data_children_debt) # Наглядная форма подачи. Есть исключение- семьи с 3 детьми более ответственны.

,count,mean,sum,share_deptors
children,,,,
0,14091,0.075438,1063,7.543822
1,4855,0.091658,445,9.165808
2,2052,0.094542,194,9.454191
3,330,0.081818,27,8.181818
4,41,0.097561,4,9.756098
5,9,0.000000,0,0.000000
20,76,0.105263,8,10.526316


2) Есть ли зависимость между семейным положением и возвратом кредита в срок?

Самые лучшие клиенты- овдовевшие и в разводе. Незарегистрировавшие отношения так же неответственно относятся к кредиту.

In [29]:
data_family_debt = data.pivot_table(index='family_status', values='debt') # Без aggfunc считается среднеарифметическое 
display(data_family_debt)                                                 # значение- mean

,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452


3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Лучшие плательщики- категории D с зарплатой 30001–50000 рублей. Самая рискованная категория- Е с минимальным доходом.

In [30]:
data_income_debt = data.pivot_table(index='total_income_category', values='debt')
display(data_income_debt)

,debt
total_income_category,
A,0.080000
B,0.070621
C,0.084915
D,0.060000
E,0.090909


4) Как разные цели кредита влияют на его возврат в срок?

Лучше всего возвращают кредит с операций с недвижимостью. 

In [31]:
data_purpose_debt = data.pivot_table(index='purpose_category', values='debt')
display(data_purpose_debt)

,debt
purpose_category,
операции с автомобилем,0.093590
операции с недвижимостью,0.072334
получение образования,0.092200
проведение свадьбы,0.080034


##### Вывод 1:

Наблюдается зависимость должников от количества детей, семейного положения и цели кредита.

## Общий вывод:

Получила таблицу с пропусками, аномалиями и множеством дубликатов. 
Пропуски восстановлены медианным значением. Дубликаты отсортированы и приведены к необходимому виду. Аномалии исправлены. 
Получилась чистая таблица. Были созданы два новых датафрема, как словари, с которыми можно что-то сделать.

Что могу сказать заказчику?

Есть рекомендация самого лучшего клиента для банка - это без детей, в разводе или овдовевший и с уровнем дохода категории D, с целью- недвижимости.
И самого невыгодного клиента- это многодетный (4 и более детей), не оформивший отношения, с минимальным уровнем дохода до 30000 руб, с целью- автомобиль.